In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [2]:
#載入資料集
import json
rating_df=pd.read_json('./userstar.json',lines=True)
rating_df

,書名,ISBN,words,USER,CONTENT,USERSTAR
0,瞄過一眼就忘不了的世界史：高中老師╳神級YouTuber 2，000萬次點閱的超人氣課程,9789863844433,瞄過一眼忘不世界史高中老師神級 萬次點閱超人氣課程高中老師神級最強圖像記憶法萬次點閱超人氣課...,Frankie,書中先按時期分類，將世界史分開前、中、後三個時期，再將前期世界史按地區分類，令錯綜複雜的大混...,5
1,大戰略：耶魯大學長紅20年大師課程，從歷史提煉的領導決策心法,9789570856712,大戰略耶魯大學長紅年大師課程從歷史提煉領導決策心法金石堂強力推薦書 耶魯大學長紅年戰略大師課...,luckylife8957,這本書花了一點時間才看完，雖然厚度頗高，讀起來卻是趣味十足。\n從一個寓言故事開始破題，然後...,5
2,大戰略：耶魯大學長紅20年大師課程，從歷史提煉的領導決策心法,9789570856712,大戰略耶魯大學長紅年大師課程從歷史提煉領導決策心法金石堂強力推薦書 耶魯大學長紅年戰略大師課...,suliwen,列舉了貫穿古今的戰役歷史，不懂戰略的人來看也是很好看的歷史故事集。,5
3,西方憑什麼：五萬年人類大歷史，破解中國落後之謎,9789868271272,西方憑什麼五萬年人類大歷史破解中國落後之謎 美國筆會書獎德國國際財經書獎 美國英國中國德國暢...,p26842082001,古早決定論 vs. 一時碰巧論,3
4,西方憑什麼：五萬年人類大歷史，破解中國落後之謎,9789868271272,西方憑什麼五萬年人類大歷史破解中國落後之謎 美國筆會書獎德國國際財經書獎 美國英國中國德國暢...,rex.mcchiu,東西方比對發展軌跡的論述，清晰、具說服力時之所需支撐社會發展方向+地理決定論,5
...,...,...,...,...,...,...
57636,城市獵人35,9789571305943,城市獵人作者簡介北條司 年 月 生本福剛縣人 年投稿第 屆手塚賞為尊崇本漫畫大師手塚治...,萌太,翻一翻城市獵人出版日期，1985，僅僅比我小一歲。在那個年代裡，相信這樣畫風精緻、兼具戰鬥跟...,3
57637,怪奇檔案全,9789573464747,怪奇檔案全恐怖漫畫界巨匠野出志早期留下短篇４篇以被譽為最佳傑作藏六怪病開始每一篇扣人心弦非常...,fevlaner,市面僅見的日野日出志作品\n也是第一本接觸，卻驚為天人的精選集\n日野日出志的作品風格不一\...,5
57638,阿保的童話,9789576930041,阿保童話如詩般阿保漫畫像夢般童年寄情歲孩童珍寶也成年人心中精靈每故事輕易帶領夢一般美麗天空中...,凜,〈炎夏裡的綿綿雪花〉是我的最愛。,5
57639,小鎮人家,9789576930058,小鎮人家阿保美代漫畫有散文氣質且精簡哲理散文觀賞小鎮人家有如閱讀莊子書中寓言故事蒲公英訊息中...,凜,純真。,5


In [3]:
# 檢查空值
rating_df.isna().any()

書名          False
ISBN        False
words       False
USER        False
CONTENT     False
USERSTAR    False
dtype: bool

In [4]:
# 清理空值並檢查空值
rating_df=rating_df.dropna(axis=0)
rating_df.isna().any()

書名          False
ISBN        False
words       False
USER        False
CONTENT     False
USERSTAR    False
dtype: bool

In [5]:
# 清理重複
duplicates = rating_df.duplicated()

if duplicates.sum() > 0:
    print('> {} duplicates'.format(duplicates.sum()))
    rating_df = rating_df[~duplicates]

print('> {} duplicates'.format(rating_df.duplicated().sum()))

> 33 duplicates
> 0 duplicates


In [6]:
print('Rating Dataframe shape : ', rating_df.shape)

Rating Dataframe shape :  (57608, 6)


In [7]:
# 移除 CONTENT 欄位
rating_df.drop(['CONTENT'], axis = 1, inplace = True)

In [9]:
# ISBN轉為字串 移除 ISBN等於 0
rating_df['ISBN'] = rating_df['ISBN'].astype('str')
rating_df=rating_df.drop(rating_df[rating_df["ISBN"]=='0'].index,axis=0) 

In [10]:
# 給user_id
import statsmodels.api as sm
rating_df['user_id'] = pd.Categorical(rating_df.USER).codes   
rating_df.head()

,書名,ISBN,words,USER,USERSTAR,user_id
0,瞄過一眼就忘不了的世界史：高中老師╳神級YouTuber 2，000萬次點閱的超人氣課程,9789863844433,瞄過一眼忘不世界史高中老師神級 萬次點閱超人氣課程高中老師神級最強圖像記憶法萬次點閱超人氣課...,Frankie,5,534
1,大戰略：耶魯大學長紅20年大師課程，從歷史提煉的領導決策心法,9789570856712,大戰略耶魯大學長紅年大師課程從歷史提煉領導決策心法金石堂強力推薦書 耶魯大學長紅年戰略大師課...,luckylife8957,5,5071
2,大戰略：耶魯大學長紅20年大師課程，從歷史提煉的領導決策心法,9789570856712,大戰略耶魯大學長紅年大師課程從歷史提煉領導決策心法金石堂強力推薦書 耶魯大學長紅年戰略大師課...,suliwen,5,6827
3,西方憑什麼：五萬年人類大歷史，破解中國落後之謎,9789868271272,西方憑什麼五萬年人類大歷史破解中國落後之謎 美國筆會書獎德國國際財經書獎 美國英國中國德國暢...,p26842082001,3,5742
4,西方憑什麼：五萬年人類大歷史，破解中國落後之謎,9789868271272,西方憑什麼五萬年人類大歷史破解中國落後之謎 美國筆會書獎德國國際財經書獎 美國英國中國德國暢...,rex.mcchiu,5,6138


In [11]:
book_features_df = rating_df.pivot_table(index = 'ISBN',columns = 'user_id',values = 'USERSTAR')
book_features_df.head()

user_id,0,1,2,3,4,5,6,7,8,9,...,8877,8878,8879,8880,8881,8882,8883,8884,8885,8886
ISBN,,,,,,,,,,,,,,,,,,,,,
1509092070025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1611172070020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1706192070010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006172050018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020070100002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
missing_value = book_features_df.isna().sum()
missing_value

user_id
0       28776
1       28772
2       28745
3       28779
4       28765
        ...  
8882    28749
8883    28778
8884    28778
8885    28777
8886    28773
Length: 8887, dtype: int64

In [13]:
rows = book_features_df.shape[0]
cols = book_features_df.shape[1]

count_empty_or_zero_cells = book_features_df.isna().sum().sum()
total_elements = rows * cols

sparsity_of_matrix = count_empty_or_zero_cells/total_elements

print('Total Empty cells are : ', count_empty_or_zero_cells)
print('Total cells in Matrix are : ', total_elements)
print('Sparsity of Matrix are : ', sparsity_of_matrix)

Total Empty cells are :  255710275
Total cells in Matrix are :  255767860
Sparsity of Matrix are :  0.999774854432453


In [14]:
# 空值用 0 取代
book_features_df.fillna(0, inplace = True)

In [15]:
book_features_df.values

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
from scipy.sparse import csr_matrix
book_features_df_matrix = csr_matrix(book_features_df.values)

In [57]:
# 建立餘弦相似模型 (K-近鄰演算法)
from sklearn.neighbors import NearestNeighbors
nearest_neighbor_model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
nearest_neighbor_model.fit(book_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [58]:
# 隨機選擇一本書
total_no_of_books = book_features_df.shape[0]
print('Total books in our pivot table : ', total_no_of_books)
random_book_index = np.random.choice(total_no_of_books)
print('Random book Index : ', random_book_index)

Total books in our pivot table :  28780
Random book Index :  3190


In [59]:
book_features_df.iloc[random_book_index]

user_id
0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
8882    0.0
8883    0.0
8884    0.0
8885    0.0
8886    0.0
Name: 9789571369792, Length: 8887, dtype: float64

In [60]:
# 向量顯示 random_book
one_dimensional_representation_of_book_vector = book_features_df.iloc[random_book_index].values.reshape(1, -1)
one_dimensional_representation_of_book_vector

array([[0., 0., 0., ..., 0., 0., 0.]])

In [61]:
# 相似書本與 random_book 的餘弦距離指數
distances, indices = nearest_neighbor_model.kneighbors(one_dimensional_representation_of_book_vector, n_neighbors = 6)
print('Distance :', distances)
print('Indices :', indices)

Distance : [[0.         0.33041052 0.5582739  0.63933215 1.         1.        ]]
Indices : [[ 3190  3192 17785 16323 19193 19186]]


In [62]:
indices = indices.flatten()
distances = distances.flatten()
for i in range(0, len(indices)):
    ## same book
    if i == 0:
        print('Recommendations for {0}:\n'.format(book_features_df.index[random_book_index]))
    else:
        ## similar books
        print('{0}: {1}, with distance of {2}'.format(i, book_features_df.index[indices[i]], distances[i]))

Recommendations for 9789571369792:

1: 9789571369815, with distance of 0.33041051999302484
2: 9789863616351, with distance of 0.5582738957006139
3: 9789863208525, with distance of 0.639332146133027
4: 9789864659142, with distance of 1.0
5: 9789864658244, with distance of 1.0


In [23]:
# 找出全部書本的相似資料集 
my_dict = {}
for book_index in range(book_features_df.shape[0]):
    
    one_dimensional_representation_of_book_vector = book_features_df.iloc[book_index].values.reshape(1, -1)
    distances, indices = nearest_neighbor_model.kneighbors(one_dimensional_representation_of_book_vector, n_neighbors = 4)
    indices = indices.flatten()
    distances = distances.flatten()
    similar_books = []
    
    for i in range(0, len(indices)):
        ## same book
        if i == 0:
            original_book = book_features_df.index[book_index]
        else:
            ## similar books
            similar_books.append(book_features_df.index[indices[i]])

        my_dict[original_book] = similar_books

In [24]:
# 秀出前三本相似
recommended_book_df = pd.DataFrame(my_dict)
recommended_book_df = recommended_book_df.T
recommended_book_df.columns = ['1st_Similar_book', '2nd_Similar_book', '3rd_Similar_book']
recommended_book_df.head()

,1st_Similar_book,2nd_Similar_book,3rd_Similar_book
1509092070025,9789864820894,9789865987916,9789865660406
1611172070020,9789866237218,9789865625061,9789861659459
1706192070010,9789865945374,9789863193678,9789863842477
2006172050018,9789865945374,9789863193678,9789863842477
2020070100002,9789865945374,9789863193678,9789863842477


In [25]:
# 書本推薦
book_name = "9789862134467"
result = recommended_book_df.loc[book_name, : ]
print('I have book recommendatation for you: \n')
for books in list(result.values):
      print(books)

I have book recommendatation for you: 

9789570839913
9789862724743
9789570839531


In [26]:
# 秀出前三本相似
recommended_book_df = pd.DataFrame(my_dict)
recommended_book_df = recommended_book_df.T
recommended_book_df.columns = ['1st_Similar_book', '2nd_Similar_book', '3rd_Similar_book']
recommended_book_df.head()

,1st_Similar_book,2nd_Similar_book,3rd_Similar_book
1509092070025,9789864820894,9789865987916,9789865660406
1611172070020,9789866237218,9789865625061,9789861659459
1706192070010,9789865945374,9789863193678,9789863842477
2006172050018,9789865945374,9789863193678,9789863842477
2020070100002,9789865945374,9789863193678,9789863842477


In [27]:
# recommended_book_df.to_json('recommended_book_df.json',orient='records',lines=True,force_ascii=False)

In [29]:
# # 存入資料庫
# from pymongo import MongoClient
# import json

# client = MongoClient(host='10.2.14.10',port=27017)
# db = client.kingstone
# coll = db.userCF
# coll.drop()
# for k,v in my_dict.items():
    
#     kv={"ISBN":k,"list":v}
# #     print(kv)
#     coll.insert([kv])

<ipython-input-29-7cb042515916>:13: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  coll.insert([kv])
